In [61]:

import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.embeddings import Embeddings
from typing import List
from langchain_postgres import PGVectorStore, PGEngine
from models.embedding_model import embedding_engine
load_dotenv()

USER = os.getenv("db_user")
PASSWORD = os.getenv("db_password")
HOST = os.getenv("db_host")
PORT = os.getenv("db_port")
DBNAME = os.getenv("db_name")




In [62]:
folder_path = "/home/muntasirfahim/personal/JP_AI_Chatbot/data"

pdf_files = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".pdf"):
            pdf_files.append(os.path.join(root, file))

all_pages = []  # To store pages from all files

for file_path in pdf_files:
    loader = PyMuPDFLoader(file_path=file_path)

    # Load the documents for this file and append to `all_pages`
    for doc in loader.lazy_load():
        all_pages.append(doc)

# Output all collected pages
print(f"Total pages loaded: {len(all_pages)}")

Total pages loaded: 30

In [63]:
# Splitting document
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=300, 
    separators=[        
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",]
)

In [64]:
splits = text_splitter.split_documents(all_pages)
print(f"Total chunks created: {len(splits)}")

Total chunks created: 44

In [65]:
CONNECTION_STRING = (
    f"postgresql+asyncpg://{USER}:{PASSWORD}@{HOST}"
    f":{PORT}/{DBNAME}"
)
pg_engine = PGEngine.from_connection_string(url=CONNECTION_STRING)

In [66]:
# len(embedding_engine.embed_query("test"))  # Test embedding engine initialization

In [67]:

from sqlalchemy.exc import ProgrammingError

from langchain_postgres import Column

TABLE_NAME = "jp_ai_bot"
VECTOR_SIZE = 3072

try:
    await pg_engine.ainit_vectorstore_table(
        table_name=TABLE_NAME,
        vector_size=VECTOR_SIZE,
        # schema_name="japan",
        id_column=Column("langchain_id", "VARCHAR"),
        # metadata_columns=[
        #     Column("likes", "INTEGER"),
        #     Column("location", "TEXT"),
        #     Column("topic", "TEXT"),
        # ],
    )
except ProgrammingError:
    # Catching the exception here
    print("Table already exists. Skipping creation.")

In [68]:

async def create_vector_store(
    table_name: str, schema_name: str = "public"
) -> PGVectorStore:
    
    vector_store = await PGVectorStore.create(
        engine=pg_engine,
        schema_name=schema_name,
        embedding_service=embedding_engine,
        table_name=table_name,
    )
    print("PGVector Store is loaded.")
    return vector_store


vector_store = await create_vector_store(
    table_name="jp_ai_bot",
    # schema_name="japan",
)

PGVector Store is loaded.

In [69]:
# push embedding to collection
print("Adding documents to vector store...")
for i in range(0, len(splits), 3):
    chunk = splits[i : i + 3]
    try:
        # Add the chunk to the vector store
        await vector_store.aadd_documents(documents=chunk)
        print(f"Chunk {i // 3 + 1}/{(len(splits) + 3) // 3} added successfully")
    except Exception as e:
        print(f"Error adding chunk {i // 3}: {e}")
        continue

Adding documents to vector store...

Chunk 1/15 added successfully

Chunk 2/15 added successfully

Chunk 3/15 added successfully

Chunk 4/15 added successfully

...

Chunk 11/15 added successfully

Chunk 12/15 added successfully

Chunk 13/15 added successfully

Chunk 14/15 added successfully

Chunk 15/15 added successfully

In [70]:
query = "Application form for the Small Business Sustainability Subsidy Program"
docs = await vector_store.asimilarity_search(query)

for doc in docs:
    print(repr(doc))

Document(id='251c315a-3215-4408-870c-9bc8e0ae7a7a', metadata={'producer': 'Microsoft® Word 2019', 'creator': 
'Microsoft® Word 2019', 'creationdate': '2024-02-07T11:52:19+09:00', 'source': 
'/home/muntasirfahim/personal/JP_AI_Chatbot/data/demo.pdf', 'file_path': 
'/home/muntasirfahim/personal/JP_AI_Chatbot/data/demo.pdf', 'total_pages': 30, 'format': 'PDF 1.7', 'title': '', 
'author': '會田 裕登', 'subject': '', 'keywords': '', 'moddate': '2024-02-07T11:52:19+09:00', 'trapped': '', 
'modDate': "D:20240207115219+09'00'", 'creationDate': "D:20240207115219+09'00'", 'page': 16}, page_content='16 \n 
\n記入日：    年 月 日 \n \n全国商工会連合会 会長 殿 \n郵便番号 \n住  所 \n名  称 \n代表者の役職・氏名          印 
\n \n \n \n小規模事業者持続化補助金交付申請書 \n \n 
小規模事業者持続化補助金＜一般型＞交付規程第６条第１項の規定に基づき、上記補助金の\n交付について、下記のとおり申請
します。 \n（注）２、５、６のみ漏れなくご記入ください \n記 \n１．補助事業の目的および内容 \n   
補助事業計画書のとおり \n＊補助事業計画書は、全国商工会連合会が指定する様式を使用すること。以下同様。 \n 
\n２．補助事業の開始日および完了予定日 \n 交付決定日 ～   年  月  日 \n \n３．補助対象経費 \n   
補助事業計画書のとおり \n \n４．補助金交付申請額 \n補助事業計画書のとおり \n 
\n５．補助事業に関して生ずる収入金に関する事項（該当するものに○） \n （１）あり ／ （２）なし \n  
\n＊「（１）あり」の場合は以下に該当事項をご記入ください。（詳細は別紙「参考資料」参照。） \n  該当事項：          
\n \n６．消費税の適用に関する事項（該当するもの１つに○） \n   （１）課税事業者 ／ （２）免税事業者 ／ 
（３）簡易課税事業者 ／ （4）２割特例（予定含む） 
\n＊消費税の区分によって、補助対象経費の算定方法が異なります。詳細は別紙「参考資料」の「消費税等仕入\n控除税額」を
参照。 \n＊２割特例が適用される対象者は、令和５年10 月１日から令和８年９月30 
日までの日の属する各課税期間に\n事業が完了する事業者となります。 \n \n \n \n \n（様式５・交付規程様式第１） 
\n【全国商工会連合会提出用（第15 回用）】')

Document(id='6576bbe1-927a-448d-a002-8349647a5be0', metadata={'producer': 'Microsoft® Word 2019', 'creator': 
'Microsoft® Word 2019', 'creationdate': '2024-02-07T11:52:19+09:00', 'source': 
'/home/muntasirfahim/personal/JP_AI_Chatbot/data/demo.pdf', 'file_path': 
'/home/muntasirfahim/personal/JP_AI_Chatbot/data/demo.pdf', 'total_pages': 30, 'format': 'PDF 1.7', 'title': '', 
'author': '會田 裕登', 'subject': '', 'keywords': '', 'moddate': '2024-02-07T11:52:19+09:00', 'trapped': '', 
'modDate': "D:20240207115219+09'00'", 'creationDate': "D:20240207115219+09'00'", 'page': 1}, page_content='1 \n目次
\n \n１．全申請者が必須の提出書類 \n \n申請書類一覧 \n２ \n小規模事業者持続化補助金事業に係る申請書（様式１） \n４ 
\n経営計画書兼補助事業計画書①（様式２） \n５ \n補助事業計画書②（様式３） \n１3 \n事業支援計画書（様式４） \n１5 
\n補助金交付申請書（様式５） \n１6 \n宣誓・同意書（様式６） \n１7 
\n２．希望する枠・特例により追加的に必要となる書類一覧 \n18 \n賃金引上げ枠・賃上げ加点の申請に係る誓約書（様式７） 
\n20 \n卒業枠の申請に係る誓約書（様式８） \n22 \nインボイス特例の申請に係る宣誓・同意書（様式９－法人用） \n23 
\nインボイス特例の申請に係る宣誓・同意書（様式９－個人事業主用） \n25 
\n３．希望する加点により追加的に必要となる書類一覧 \n27 \n事業承継診断票（様式１０） \n28 \n４．その他 \n29 
\n事務所賃料関係 \n29')

Document(id='284c9ec9-6b54-4c74-8861-d4f285a0777c', metadata={'producer': 'Microsoft® Word 2019', 'creator': 
'Microsoft® Word 2019', 'creationdate': '2024-02-07T11:52:19+09:00', 'source': 
'/home/muntasirfahim/personal/JP_AI_Chatbot/data/demo.pdf', 'file_path': 
'/home/muntasirfahim/personal/JP_AI_Chatbot/data/demo.pdf', 'total_pages': 30, 'format': 'PDF 1.7', 'title': '', 
'author': '會田 裕登', 'subject': '', 'keywords': '', 'moddate': '2024-02-07T11:52:19+09:00', 'trapped': '', 
'modDate': "D:20240207115219+09'00'", 'creationDate': "D:20240207115219+09'00'", 'page': 2}, page_content='2 
\n１．全申請者が必須の提出書類 \n 申請書類一覧 \n \n書類名 \n様式 \n法人 \n個人 \nNPO \n種別 \n注意 \n事項 \n1 
\n小規模事業者持続化補助金事業に係る申請書 \n（電子申請の場合は不要） \n様式 \n1 \n○ \n○ \n○ \n原本 \n※ 
\n次頁\n参照 \n2 \n経営計画書兼補助事業計画書① \n様式 \n2 \n○ \n○ \n○ \n原本 \n3 \n補助事業計画書② \n様式 \n3 \n○ 
\n○ \n○ \n原本 \n4 \n事業支援計画書 ※ \n様式\n４ \n○ \n○ \n○ \n原本 \n5 \n補助金交付申請書 ※ 
\n（郵送による申請の場合は必要） \n様式\n５ \n○ \n○ \n○ \n原本 \n６ \n宣誓・同意書 \n様式 \n６ \n○ \n○ \n○ \n原本 
\n７ \n電子媒体（様式１、様式２、様式３、様式５、様式６、（該当者\nのみ 様式７、様式８、様式９）） ※ \n― \n○ \n○ 
\n○ \n― \n８ \n貸借対照表および損益計算書（直近１期分） ※ \n― \n○ \n― \n― \n写し \n９ \n株主名簿（該当者のみ） ※ 
\n― \n○ \n― \n― \n写し \n１０ 
\n直近の確定申告書【第一表及び第二表及び収支内訳書\n（１・２面）または第一表及び第二表及び所得税青色申告決\n算書（
１～４面）】（税務署受付印のあるもの）または開業届\n（税務署受付印のあるもの） ※ \n― \n― \n○ \n― \n写し \n1１ 
\n貸借対照表および活動計算書（直近１期分） ※ \n― \n― \n― \n○ \n写し \n1２ 
\n現在事項全部証明書または履歴事項全部証明書 \n（申請書の提出日から３か月以内の日付のもの（原本）） \n― \n― \n― \n○
\n原本 \n1３ \n法人税確定申告書（別表一（受付印のある用紙）および別\n表四（所得の簡易計算））（直近１期分） ※ \n― 
\n― \n― \n○ \n写し')

Document(id='9f1e4627-81df-4f22-b348-fff4ba6eb19f', metadata={'producer': 'Microsoft® Word 2019', 'creator': 
'Microsoft® Word 2019', 'creationdate': '2024-02-07T11:52:19+09:00', 'source': 
'/home/muntasirfahim/personal/JP_AI_Chatbot/data/demo.pdf', 'file_path': 
'/home/muntasirfahim/personal/JP_AI_Chatbot/data/demo.pdf', 'total_pages': 30, 'format': 'PDF 1.7', 'title': '', 
'author': '會田 裕登', 'subject': '', 'keywords': '', 'moddate': '2024-02-07T11:52:19+09:00', 'trapped': '', 
'modDate': "D:20240207115219+09'00'", 'creationDate': "D:20240207115219+09'00'", 'page': 4}, page_content='4 \n 
\n記入日：    年 月 日 \n全国商工会連合会 会長 殿 \n郵便番号 \n \n住所（都道府県 \n名から記載） \n \n \n名称 \n \n 
\n \n印 \n代表者の役職 \n \n代表者氏名 \n（姓／名） \n \n \n \n \n電話番号 \n \n会社代表電話番号 \n \n 
\n小規模事業者持続化補助金事業＜一般型＞に係る申請書 \n  
\n小規模事業者持続化補助金＜一般型＞の交付を受けたいので、下記の書類を添えて申請\nします。また、申請にあたっては、
公募要領に記載された「重要説明事項」（P.３～４）を確認し、\nその内容を十分に理解しています。 \n記 \n 
\n・経営計画書兼補助事業計画書①（様式２） \n・補助事業計画書②（様式３） 
\n・事業支援計画書（様式４）＊依頼に基づき、地域の商工会が作成します。 
\n・補助金交付申請書（様式５）＊補助金事務局でお預かりし、採択決定後に正式受理します。 \n・宣誓・同意書（様式６） 
\n●その他必要書類、電子媒体（ＣＤ－Ｒ・ＵＳＢメモリ等） \n◇法人（特定非営利活動法人を除く）の場合： 
\n・貸借対照表および損益計算書（直近１期分） \n◇個人事業主の場合： 
\n・直近の確定申告書（第一表及び第二表及び収支内訳書（１・２面）または第一表及び 
\n第二表及び所得税青色申告決算書（１～４面））または開業届 \n  ◇特定非営利活動法人の場合： \n   
・貸借対照表および活動計算書（直近１期分）  \n    ・現在事項全部証明書または履歴事項全部証明書 \n     
・法人税確定申告書（直近１期分） \n \n \n \n \n \n \n \n \n \n \n（様式１） \n【全国商工会連合会提出用（第15 
回用）】')

In [71]:
from rich import print
query_vector = embedding_engine.embed_query(query)
docs = await vector_store.asimilarity_search_by_vector(query_vector, k=2)
print(docs)

[
    Document(
        id='251c315a-3215-4408-870c-9bc8e0ae7a7a',
        metadata={
            'producer': 'Microsoft® Word 2019',
            'creator': 'Microsoft® Word 2019',
            'creationdate': '2024-02-07T11:52:19+09:00',
            'source': '/home/muntasirfahim/personal/JP_AI_Chatbot/data/demo.pdf',
            'file_path': '/home/muntasirfahim/personal/JP_AI_Chatbot/data/demo.pdf',
            'total_pages': 30,
            'format': 'PDF 1.7',
            'title': '',
            'author': '會田 裕登',
            'subject': '',
            'keywords': '',
            'moddate': '2024-02-07T11:52:19+09:00',
            'trapped': '',
            'modDate': "D:20240207115219+09'00'",
            'creationDate': "D:20240207115219+09'00'",
            'page': 16
        },
        page_content='16 \n \n記入日：    年 月 日 \n \n全国商工会連合会 会長 殿 \n郵便番号 \n住  所 \n名  称 
\n代表者の役職・氏名          印 \n \n \n \n小規模事業者持続化補助金交付申請書 \n \n 
小規模事業者持続化補助金＜一般型＞交付規程第６条第１項の規定に基づき、上記補助金の\n交付について、下記のとおり申請
します。 \n（注）２、５、６のみ漏れなくご記入ください \n記 \n１．補助事業の目的および内容 \n   
補助事業計画書のとおり \n＊補助事業計画書は、全国商工会連合会が指定する様式を使用すること。以下同様。 \n 
\n２．補助事業の開始日および完了予定日 \n 交付決定日 ～   年  月  日 \n \n３．補助対象経費 \n   
補助事業計画書のとおり \n \n４．補助金交付申請額 \n補助事業計画書のとおり \n 
\n５．補助事業に関して生ずる収入金に関する事項（該当するものに○） \n （１）あり ／ （２）なし \n  
\n＊「（１）あり」の場合は以下に該当事項をご記入ください。（詳細は別紙「参考資料」参照。） \n  該当事項：          
\n \n６．消費税の適用に関する事項（該当するもの１つに○） \n   （１）課税事業者 ／ （２）免税事業者 ／ 
（３）簡易課税事業者 ／ （4）２割特例（予定含む） 
\n＊消費税の区分によって、補助対象経費の算定方法が異なります。詳細は別紙「参考資料」の「消費税等仕入\n控除税額」を
参照。 \n＊２割特例が適用される対象者は、令和５年10 月１日から令和８年９月30 
日までの日の属する各課税期間に\n事業が完了する事業者となります。 \n \n \n \n \n（様式５・交付規程様式第１） 
\n【全国商工会連合会提出用（第15 回用）】'
    ),
    Document(
        id='6576bbe1-927a-448d-a002-8349647a5be0',
        metadata={
            'producer': 'Microsoft® Word 2019',
            'creator': 'Microsoft® Word 2019',
            'creationdate': '2024-02-07T11:52:19+09:00',
            'source': '/home/muntasirfahim/personal/JP_AI_Chatbot/data/demo.pdf',
            'file_path': '/home/muntasirfahim/personal/JP_AI_Chatbot/data/demo.pdf',
            'total_pages': 30,
            'format': 'PDF 1.7',
            'title': '',
            'author': '會田 裕登',
            'subject': '',
            'keywords': '',
            'moddate': '2024-02-07T11:52:19+09:00',
            'trapped': '',
            'modDate': "D:20240207115219+09'00'",
            'creationDate': "D:20240207115219+09'00'",
            'page': 1
        },
        page_content='1 \n目次 \n \n１．全申請者が必須の提出書類 \n \n申請書類一覧 \n２ 
\n小規模事業者持続化補助金事業に係る申請書（様式１） \n４ \n経営計画書兼補助事業計画書①（様式２） \n５ 
\n補助事業計画書②（様式３） \n１3 \n事業支援計画書（様式４） \n１5 \n補助金交付申請書（様式５） \n１6 
\n宣誓・同意書（様式６） \n１7 \n２．希望する枠・特例により追加的に必要となる書類一覧 \n18 
\n賃金引上げ枠・賃上げ加点の申請に係る誓約書（様式７） \n20 \n卒業枠の申請に係る誓約書（様式８） \n22 
\nインボイス特例の申請に係る宣誓・同意書（様式９－法人用） \n23 
\nインボイス特例の申請に係る宣誓・同意書（様式９－個人事業主用） \n25 
\n３．希望する加点により追加的に必要となる書類一覧 \n27 \n事業承継診断票（様式１０） \n28 \n４．その他 \n29 
\n事務所賃料関係 \n29'
    )
]